In [1]:
import herringbone as hb

All initialization tests passed.
imported herringbone without any errors :)


In [ ]:

# create an MDP
map_names = ["-", "example", "easy", "danger_holes", "double_fish", "wall_of_death", "example2", "mega"]
selected_map_id = 1

state_path = "herringbone/env_core/config/state_config.json"
map_path = f"herringbone/env_core/maps/{map_names[selected_map_id]}.csv"
action_path = "herringbone/env_core/config/action_config.json"

GAMMA = 0.9

demo_mdp = hb.MDP(state_path, map_path, action_path, seed=42, gamma=GAMMA)


In [3]:

random_policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)
episode = hb.Episode(mdp=demo_mdp, policy=random_policy, max_depth=1000)
episode.peek()
episode.run("sar")
print(episode.trajectory)

╔═══════╦═══════╦═══════╦═══════╦═══════╗
║   -1  ║   -1  ║   -1  ║   -1  ║  -100 ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║   -1  ║   -1  ║   -1  ║   -1  ║   -1  ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║   -1  ║   -1  ║  -100 ║   -1  ║   -1  ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║  -100 ║   -1  ║   -1  ║   -1  ║   -1  ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║   -1  ║   -1  ║   -1  ║   -1  ║   10  ║
╚═══════╩═══════╩═══════╩═══════╩═══════╝[0]
t: 0 | S: [3, 3], R: -1, A: ↑
t: 1 | S: [2, 3], R: -1, A: ↓
t: 2 | S: [3, 3], R: -1, A: ↑
t: 3 | S: [2, 3], R: -1, A: ↑
t: 4 | S: [1, 3], R: -1, A: ↑
t: 5 | S: [0, 3], R: -1, A: ↓
t: 6 | S: [1, 3], R: -1, A: ↑
t: 7 | S: [0, 3], R: -1, A: →
t: 8 | S: [0, 4], R: -100, A: None
Trajectory(states=[[3, 3], [2, 3], [3, 3], [2, 3], [1, 3], [0, 3], [1, 3], [0, 3], [0, 4]], actions=[↑, ↓, ↑, ↑, ↑, ↓, ↑, →], rewards=[nan, -1, -1, -1, -1, -1, -1, -1, -100])


In [4]:
# Add the render function for previewing a Value Function (hate this!)
def preview_V(mdp, learned_V):
    
    states_2d = mdp.get_board().states
    x = len(states_2d)
    y = len(states_2d[0])
    
    v_values =  ['%.2f' % v for v in list(learned_V.values())]

    two_d_list = [v_values[i * y:(i + 1) * y] for i in range(x)]

    # Print 2D list
    for row in two_d_list:
        print(row)

In [5]:
policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)  # defaults to uniform
print(policy)

╔═════════╦═════════╦═════════╦═════════╦═════════╗
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╚═════════╩═════════╩═════════╩═════════╩═════════╝


In [6]:
episode = hb.Episode(mdp=demo_mdp, policy=policy, max_depth=1000)

In [7]:
episode.run("sar")

t: 0 | S: [3, 3], R: -1, A: ←
t: 1 | S: [3, 2], R: -1, A: ↑
t: 2 | S: [2, 2], R: -100, A: None


In [8]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp)
mc_predictor.evaluate_policy(policy, n_samples=N)

In [9]:
preview_V(demo_mdp, mc_predictor.value_functions)

['-37.41', '-41.95', '-51.53', '-60.82', '0.00']
['-54.14', '-55.86', '-61.29', '-54.61', '-65.01']
['-68.47', '-74.32', '0.00', '-55.24', '-40.86']
['0.00', '-69.84', '-58.31', '-34.02', '-18.73']
['-62.23', '-52.36', '-40.21', '-19.69', '0.00']


In [10]:

N = 1000
mc_control = hb.MonteCarloController(demo_mdp, epsilon=0.25)
mc_control.train(n_episodes=N)
trained_policy = mc_control.policy

In [11]:
print(trained_policy.get_policy())
print(trained_policy)

{[0, 0]: {↑: 0.7093345261483012, ↓: 0.0943948379888203, ←: 0.09679999750470507, →: 0.09947063835817344}, [0, 1]: {↑: 0.06498524437565394, ↓: 0.06357586944290133, ←: 0.06267531489871683, →: 0.8087635712827278}, [0, 2]: {↑: 0.06983267473754859, ↓: 0.8021578430390877, ←: 0.0646296183142185, →: 0.06337986390914521}, [0, 3]: {↑: 0.7395425453360874, ↓: 0.09776607010772033, ←: 0.09953507748812199, →: 0.06315630706807034}, [0, 4]: {↑: 0.25, ↓: 0.25, ←: 0.25, →: 0.25}, [1, 0]: {↑: 0.059534930029504676, ↓: 0.7739534290126943, ←: 0.10418603438083268, →: 0.06232560657696829}, [1, 1]: {↑: 0.06315278868614109, ↓: 0.0622523634447295, ←: 0.8097532277452256, →: 0.06484162012390376}, [1, 2]: {↑: 0.1813628508273646, ↓: 0.6728053007774504, ←: 0.07629021824241065, →: 0.06954163015277437}, [1, 3]: {↑: 0.24961041531768585, ↓: 0.08145798399171397, ←: 0.06381927305995024, →: 0.60511232763065}, [1, 4]: {↑: 0.0624999999208524, ↓: 0.8124999998124419, ←: 0.06250000024600379, →: 0.06250000002070193}, [2, 0]: {↑: 0.

In [12]:
episode = hb.Episode(mdp=demo_mdp, policy=trained_policy, max_depth=1000)
episode.run("sar")

t: 0 | S: [3, 3], R: -1, A: →
t: 1 | S: [3, 4], R: -1, A: ↓
t: 2 | S: [4, 4], R: 10, A: None


In [13]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp)
mc_predictor.evaluate_policy(trained_policy, n_samples=N)

In [14]:
preview_V(demo_mdp, mc_predictor.value_functions)

['0.00', '0.00', '0.00', '0.00', '0.00']
['0.00', '0.00', '0.00', '4.58', '5.24']
['0.00', '0.00', '0.00', '1.20', '6.82']
['0.00', '4.58', '-1.00', '6.43', '9.22']
['0.00', '0.00', '6.23', '9.03', '0.00']
